# PY

In [8]:

"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: data.py : python script for data collection                                                 -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

# Load packages
import pandas as pd
import numpy as np
from os import listdir, path
from os.path import isfile, join

# Get the absolute path of the folder
abspath = path.abspath("files/")

# Read all the files within the files folder
files = [f[8:-4] for f in listdir(abspath) if isfile(join(abspath, f))]

# Chronologically ordered files list
files = ["NAFTRAC_" + i.strftime("%Y%m%d") for i in sorted(pd.to_datetime(files))]

# Read and store all the files in a dictionary
data_files = {}
for i in files:
    # Read the file
    data = pd.read_csv("files/" + i + ".csv", skiprows=2, header=0)
    # Select only not null columns
    data = data.loc[:, pd.notnull(data.columns)]
    # Clean the ticker name for later use with yfinance
    data["Ticker"] = [i.replace("*","") for i in data["Ticker"]]
    # Weight as decimal
    data["Peso (%)"] = [i/100 for i in data["Peso (%)"]]
    # Save current file in dictionary of data
    data_files[i] = data



In [9]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: functions.py : python script with general functions                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

import pandas as pd
import numpy as np
import yfinance as yf
import time
import datetime

# Get Dates
def f_dates(p_files):
    # Labels for dataframe and yfinance
    t_dates = [i.strftime("%d-%m-%Y") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # For other calculations
    i_dates = [i.strftime("%Y-%m-%d") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # Final data to return
    r_f_dates = {"i_dates": i_dates, "t_dates": t_dates}

    return r_f_dates

#%%
# Get Tickers
def f_tickers(p_archivos, p_data_archivos):
    tickers = []
    for i in p_archivos:
        l_tickers = list(p_data_archivos[i]["Ticker"])
        [tickers.append(i + ".MX") for i in l_tickers]
    global_tickers = np.unique(tickers).tolist()

    # Name adjustment
    global_tickers = [i.replace("GFREGIOO.MX", "RA.MX") for i in global_tickers]
    global_tickers = [i.replace("MEXCHEM.MX", "ORBIA.MX") for i in global_tickers]
    global_tickers = [i.replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX") for i in global_tickers]

    # Remove problematic tickers and cash entries
    [global_tickers.remove(i) for i in ["MXN.MX", "USD.MX", "KOFL.MX", "KOFUBL.MX",
                                        "BSMXB.MX", "SITESB.1.MX", "NEMAKA.MX", "NMKA.MX"]]

    return global_tickers

#%%
# Get Prices
def f_get_prices(p_tickers, p_fechas):
    # Initial date, no changes
    f_ini = p_fechas[0]

    # Initial date plus 3 days
    f_fin = str(datetime.datetime.strptime(p_fechas[-1], "%Y-%m-%d") + datetime.timedelta(days=3))[:10]

    # Time counter
    inicio = time.time()

    # Yahoo finance data download
    data = yf.download(p_tickers, start=f_ini, end=f_fin, actions=False, group_by="close",
                       interval="1d", auto_adjust=False, prepost=False, threads=True)

    # Time length of process
    tiempo = "It took", round(time.time() - inicio, 2), "seconds."

    # Morph date column
    data_close = pd.DataFrame({i: data[i]["Close"] for i in p_tickers})

    # We assume NAFRTAC rebalance and Yahoo finance close price times align.

    # Only relevant dates
    ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(p_fechas)))

    # All prices
    precios = data_close.iloc[[int(np.where(data_close.index == i)[0]) for i in ic_fechas]]

    # Order columns
    precios = precios.reindex(sorted(precios.columns), axis=1)

    return {"precios": precios, "tiempo": tiempo}


In [11]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: main.py : python script with the main functionality                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""
# %%
# Import other scripts
import data as dt
import functions as fn
import visualizations as vz

# %%
# ---- Base

# Step 1 - Read all the files
data_files = dt.data_files

# Step 2 - Get all the dates
dates = fn.f_dates(p_files=dt.files)

# Display the first 5 dates in the 2 formats
print(dates["i_dates"][0:4])
print(dates["t_dates"][0:4])
#%%
# Step 3 - Get the tickers for the calculations
global_tickers = fn.f_tickers(p_archivos=dt.files, p_data_archivos=data_files)

# Display global tickers
print(global_tickers[0:4])
#%%
# ---- Historical Prices
global_prices = fn.f_get_prices(p_tickers=global_tickers, p_fechas=dates["i_dates"])
precios = global_prices["precios"]




['2018-01-31', '2018-02-28', '2018-03-28', '2018-04-30']
['31-01-2018', '28-02-2018', '28-03-2018', '30-04-2018']
['AC.MX', 'ALFAA.MX', 'ALPEKA.MX', 'ALSEA.MX']
[*********************100%***********************]  39 of 39 completed

1 Failed download:
-  .MX: No data found, symbol may be delisted


# PASIVA

## PRE

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web

In [40]:
kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)

cash = (kay.iloc[35]["Peso (%)"]+kay.iloc[34]["Peso (%)"]+kay.iloc[16]["Peso (%)"]+kay.iloc[10]["Peso (%)"])/100*1000000


kay["Ticker"] = [i.replace("*","") for i in data["Ticker"]]

    # Weight as decimal
kay["Ticker"] = kay["Ticker"] + '.MX'
kay = kay.drop([10,16,34,35,36])
kay = kay.sort_values(by=['Ticker'])
cash = cash + sum(kay["Peso (%)"]/100*1000000*0.00125)

In [41]:
dates = fn.f_dates(p_files=dt.files)
fechi = dates["i_dates"][0:25]
startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
weight = np.array(kay["Peso (%)"] )

In [42]:
kay["Ticker"] = kay["Ticker"].replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX")
kay["Ticker"] = kay["Ticker"].replace("SITESB.1.MX", "SITESB-1.MX") 
symbols = np.array(kay["Ticker"])
price_data = web.get_data_yahoo(symbols,
                               start = startp,
                               end = fechi[-1], interval='d')
price_data = price_data["Adj Close"]
price_data.index.strftime("%Y-%m-%d")
price_data = price_data.loc[fechi]

In [43]:
ret_data = price_data.pct_change()

In [44]:
weighted_returns = (weight * ret_data)

In [45]:
port_ret = weighted_returns.sum(axis=1)/100
capital = 1000000*(1+port_ret)-cash
cumulative_ret = (port_ret+1).cumprod()-1

In [46]:
df_pasiva_a = pd.DataFrame()
df_pasiva_a["Capital"] = capital
df_pasiva_a["Rendimiento"] = port_ret
df_pasiva_a["Rendimiendo Acumulado"] = cumulative_ret
df_pasiva_a

,Capital,Rendimiento,Rendimiendo Acumulado
Date,,,
2018-01-31,9.539060e+05,0.000000,0.000000
2018-02-28,9.018408e+05,-0.052065,-0.052065
2018-03-28,9.295121e+05,-0.024394,-0.075189
2018-04-30,1.000486e+06,0.046580,-0.032111
2018-05-31,8.828742e+05,-0.071032,-0.100862
2018-06-29,1.030054e+06,0.076148,-0.032394
2018-07-31,9.919567e+05,0.038051,0.004424
2018-08-31,9.563928e+05,0.002487,0.006921
2018-09-28,9.495272e+05,-0.004379,0.002512


In [4]:
def pasive_data(symbols,  startd, endd):
    startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
    price_data = web.get_data_yahoo(symbols,
                               start = startd,
                               end = endd, interval='d')
    price_data = price_data["Adj Close"]
    price_data.index.strftime("%Y-%m-%d")
    price_data = price_data.loc[fechi]
    ret_data = price_data.pct_change()
    weighted_returns = (weight * ret_data)
    port_ret = weighted_returns.sum(axis=1)/100
    capital = 1000000*(1+port_ret)-cash
    cumulative_ret = (port_ret+1).cumprod()-1
    df_pasiva_a = pd.DataFrame()
    df_pasiva_a["Capital"] = capital
    df_pasiva_a["Rendimiento"] = port_ret
    df_pasiva_a["Rendimiendo Acumulado"] = cumulative_ret
    return df_pasiva_a

In [14]:
def tickin(start_date ,last_date ):
    kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)

    cash = (kay.iloc[35]["Peso (%)"]+kay.iloc[34]["Peso (%)"]+kay.iloc[16]["Peso (%)"]+kay.iloc[10]["Peso (%)"])/100*1000000


    kay["Ticker"] = [i.replace("*","") for i in data["Ticker"]]

        # Weight as decimal
    kay["Ticker"] = kay["Ticker"] + '.MX'
    kay = kay.drop([10,16,34,35,36])
    kay = kay.sort_values(by=['Ticker'])
    cash = cash + sum(kay["Peso (%)"]/100*1000000*0.00125)
    dates = fn.f_dates(p_files=dt.files)
    fechi = dates["i_dates"][start_date:last_date]
    weight = np.array(kay["Peso (%)"] )
    kay["Ticker"] = kay["Ticker"].replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX")
    kay["Ticker"] = kay["Ticker"].replace("SITESB.1.MX", "SITESB-1.MX") 
    symbols = np.array(kay["Ticker"])
    return symbols

In [26]:
def tickin2(start_date ,last_date ):
    kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)

    cash = (kay.iloc[35]["Peso (%)"]+kay.iloc[34]["Peso (%)"]+kay.iloc[16]["Peso (%)"]+kay.iloc[10]["Peso (%)"])/100*1000000


    kay["Ticker"] = [i.replace("*","") for i in data["Ticker"]]

        # Weight as decimal
    kay["Ticker"] = kay["Ticker"] + '.MX'
    kay = kay.drop([10,16,34,35,36])
    kay = kay.sort_values(by=['Ticker'])
    cash = cash + sum(kay["Peso (%)"]/100*1000000*0.00125)
    dates = fn.f_dates(p_files=dt.files)
    fechi = dates["i_dates"][start_date:last_date]
    weight = np.array(kay["Peso (%)"] )
    kay["Ticker"] = kay["Ticker"].replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX")
    kay["Ticker"] = kay["Ticker"].replace("SITESB.1.MX", "SITESB-1.MX") 
    symbols = np.array(kay["Ticker"])   
    startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
    price_data = web.get_data_yahoo(symbols,
                               start = startp,
                               end = fechi[-1], interval='d')
    price_data = price_data["Adj Close"]
    price_data.index.strftime("%Y-%m-%d")
    price_data = price_data.loc[fechi]
    ret_data = price_data.pct_change()
    weighted_returns = (weight * ret_data)
    port_ret = weighted_returns.sum(axis=1)/100
    capital = 1000000*(1+port_ret)-cash
    cumulative_ret = (port_ret+1).cumprod()-1
    df_pasiva = pd.DataFrame()
    df_pasiva["Capital"] = capital
    df_pasiva["Rendimiento"] = port_ret
    df_pasiva["Rendimiendo Acumulado"] = cumulative_ret
    return df_pasiva

In [22]:
b_e = tickin2(0,25)
b_e

,Capital,Rendimiento,Rendimiendo Acumulado
Date,,,
2018-01-31,9.539060e+05,0.000000,0.000000
2018-02-28,9.018408e+05,-0.052065,-0.052065
2018-03-28,9.295121e+05,-0.024394,-0.075189
2018-04-30,1.000486e+06,0.046580,-0.032111
2018-05-31,8.828742e+05,-0.071032,-0.100862
2018-06-29,1.030054e+06,0.076148,-0.032394
2018-07-31,9.919567e+05,0.038051,0.004424
2018-08-31,9.563928e+05,0.002487,0.006921
2018-09-28,9.495272e+05,-0.004379,0.002512


In [25]:
b_e = tickin2(25,39)
b_e

,Capital,Rendimiento,Rendimiendo Acumulado
Date,,,
2020-02-28,9.539060e+05,0.000000,0.000000
2020-03-31,8.120022e+05,-0.141904,-0.141904
2020-04-30,1.022605e+06,0.068699,-0.082953
2020-05-29,9.498217e+05,-0.004084,-0.086699
2020-06-30,1.012228e+06,0.058322,-0.033433
2020-07-31,9.524649e+05,-0.001441,-0.034826
2020-08-31,9.607853e+05,0.006879,-0.028186
2020-09-30,9.756084e+05,0.021702,-0.007095
2020-10-30,9.537261e+05,-0.000180,-0.007274


In [38]:
def dframe (capital, port_ret, cumulative_ret):
    df_pasiva_a = pd.DataFrame()
    df_pasiva_a["Capital"] = capital
    df_pasiva_a["Rendimiento"] = port_ret
    df_pasiva_a["Rendimiendo Acumulado"] = cumulative_ret
    return df_pasiva_a

In [136]:
pasive_data(symbols,  tickin(25,39)[0], tickin(25,39)[-1]).head(3)

Symbols,AC.MX,ALFAA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BOLSAA.MX,CEMEXCPO.MX,CUERVO.MX,ELEKTRA.MX,...,LIVEPOLC-1.MX,MEGACPO.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,SITESB-1.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-31,-0.193742,-0.666476,-0.260291,-1.201953,-0.762514,-0.206397,-0.062929,-1.654638,-0.032372,0.154331,...,-0.139558,-0.012847,-0.573448,-0.138895,-0.231453,-0.127405,0.016318,-1.606878,-0.105860,0.094757
2020-04-30,-0.007318,0.735250,0.129763,0.496761,0.196961,-0.044805,0.101948,0.211278,0.272515,-0.026984,...,0.055260,-0.061534,0.160330,0.225057,0.069879,0.290133,0.020891,-0.384428,0.178485,0.378694


In [134]:
dframeb (capital, port_ret, cumulative_ret)

,Capital,Rendimiento,Rendimiendo Acumulado
Date,,,
2018-01-31,9.539060e+05,0.000000,0.000000
2018-02-28,9.018408e+05,-0.052065,-0.052065
2018-03-28,9.295121e+05,-0.024394,-0.075189
2018-04-30,1.000486e+06,0.046580,-0.032111
2018-05-31,8.828742e+05,-0.071032,-0.100862
2018-06-29,1.030054e+06,0.076148,-0.032394
2018-07-31,9.919567e+05,0.038051,0.004424
2018-08-31,9.563928e+05,0.002487,0.006921
2018-09-28,9.495272e+05,-0.004379,0.002512


## POST